In [18]:
from traffic_generator import traffic_generator
import threading
import time

In [ ]:
"""
关于数据格式
逻辑拓扑应该呈现为有向环形, 表现为:
TOPOLOGY = ["Node_i", "Node_j", "Node_k", "Node_l", "Node_m"]
每个Node的Segment的数据量依照DICT方式给出, 表现为:
DATA_DICT = {
    "Node_i":[10, 20, 30, 40, 1],
    "Node_j":[15, 40, 25, 35, 1],
    "Node_k":[12, 18, 40, 25, 1],
    "Node_l":[40, 12, 15, 23, 1],
    "Node_m":[20, 13, 25, 11, 12]
}
"""

In [19]:
TOPOLOGY = ["Node_i", "Node_j", "Node_k", "Node_l", "Node_m"]
DATA_DICT = {
    "Node_i":[10, 20, 30, 40, 1],
    "Node_j":[15, 40, 25, 35, 1],
    "Node_k":[12, 18, 40, 25, 1],
    "Node_l":[40, 12, 15, 23, 1],
    "Node_m":[20, 13, 25, 11, 12]
}
link_bandwidth = 2 # 应该以邻接矩阵给出
traffic_distribution = "uniform"
link_delay = 5

In [ ]:
# Ring-AllReduce_algorithms

NODES_NUM = len(TOPOLOGY)
MAX_STEPS = 2 * (NODES_NUM - 1)
FINAL_DICT = DATA_DICT
for step in range(MAX_STEPS):
    REDUCE_FLAG = True
    if step >= (NODES_NUM - 1):
        REDUCE_FLAG = False
    for node_id in range(NODES_NUM):
        data_index = ((node_id + step*(NODES_NUM-1)) % NODES_NUM)
        # print(data_index)
        if node_id == NODES_NUM-1:
            src_node = TOPOLOGY[node_id]
            dst_node = TOPOLOGY[0]
            if REDUCE_FLAG:
                FINAL_DICT[dst_node][data_index] += FINAL_DICT[src_node][data_index]
                continue
            FINAL_DICT[dst_node][data_index] = FINAL_DICT[src_node][data_index]
            continue
        src_node = TOPOLOGY[node_id]
        dst_node = TOPOLOGY[node_id+1]
        if REDUCE_FLAG:
            FINAL_DICT[dst_node][data_index] += FINAL_DICT[src_node][data_index]
            continue
        FINAL_DICT[dst_node][data_index] = FINAL_DICT[src_node][data_index]

print(FINAL_DICT)

{'Node_i': [97, 103, 135, 134, 16], 'Node_j': [97, 103, 135, 134, 16], 'Node_k': [97, 103, 135, 134, 16], 'Node_l': [97, 103, 135, 134, 16], 'Node_m': [97, 103, 135, 134, 16]}


In [ ]:
# Broadcast_algorithms (Based on Binary-Tree)

father_node_list = ["Node_i"]
son_node_list = ["node_j", "node_k", "node_l", "node_m", "node_n"]
iter_cnt = 0
for son_index, son_node in enumerate(son_node_list):
    for father_index, father_node in enumerate(father_node_list):
        father_node SEND TO son_node
        son_node_list DELETE son_node
        father_node_list ADD son_node
        break
        


580.0
{'Node_i': [97, 103, 135, 134, 16], 'Node_j': [97, 103, 135, 134, 16], 'Node_k': [97, 103, 135, 134, 16], 'Node_l': [97, 103, 135, 134, 16], 'Node_m': [97, 103, 135, 134, 16]}
